In [69]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import os
from matplotlib import pyplot as plt

import nibabel as nib

In [70]:
# Formulate train, val, test file paths for "scans" and "labels"
# Stored as filepaths, as the generator will do the file reading
def get_nifti_files_in(directory):
    paths = []
    for file in os.listdir(directory):
        if file.endswith(".nii.gz"):
            paths.append(os.path.join(directory, file))
    
    # Sort so ordering is not file system order dependent
    return sorted(paths)

base_path = "data"
#data_dimensions = (256, 256, 128)
data_dimensions = (32, 32, 32)

# Background, Body, Bone, Bladder, Rectum, Prostate
class_count = 6

scans = get_nifti_files_in(os.path.join(base_path, "semantic_MRs_anon"))
labels = get_nifti_files_in(os.path.join(base_path, "semantic_labels_anon"))

# Make sure we have an equal number of scans and labels
assert len(scans) == len(labels)

# Zips the two "scans" and "labels" arrays together to produce [[scan_filename, label_filename], ...]
data_paths = np.dstack((scans, labels))[0]

print("Raw Labelled Scans: " + str(len(data_paths)))

Raw Labelled Scans: 211


In [88]:
# Categorise input data into a form which represents the patient heirarchy, to not train and test on the same patient!
def generate_patient_data_heirarchy(data_paths):
    # Just incase patient IDs aren't continuous, i will use a dictionary mapping from id -> [[scan_path, label_path]]
    patient_data = {}
    
    for scan_path, label_path in data_paths:
        # Use scan filename to parse out patient ID
        scan_file = os.path.basename(scan_path)
        # Using scan formatting of "Case_[patient_id]_..."
        patient_id = int(scan_file.split("_")[1])
        
        # Simple dict, array population with first time checking
        if patient_id in patient_data:
            patient_data[patient_id].append([scan_path, label_path])
        else:
            patient_data[patient_id] = [[scan_path, label_path]]
        
    return patient_data

patient_data = generate_patient_data_heirarchy(data_paths)

print("Patient Count: " + str(len(patient_data.keys())))

Patient Count: 38


In [97]:
# Create Train, Val, Test split (move into utility function)
def train_val_test_split(train_split, val_split, test_split, patient_data):
    print("Splitting dataset using Train: " + str(train_split) + 
          " Val: " + str(val_split) + " Test: " + str(test_split))

    assert train_split + val_split + test_split == 1.0

    patient_ids = patient_data.keys()
    # Randomise the order in which we loop over the data - gives a more random distribution of data in each bucket
    np.random.shuffle(data_paths)
    
    train_data = []
    val_data = []
    test_data = []
    
    # The method works by calculating the percent allocated after each allocation,
    # Then assigning more data to the bucket which is FURTHEST BELOW the desired
    # Allocation. This iterative method will give us a good allocation, which is close enough to the desired one.
    for patient_id in patient_ids:
        # Clamp to minimum value of 1 so we don't have to handle div by 0 on first iteration
        assigned_count = max(len(train_data) + len(val_data) + len(test_data), 1)
        
        # Calculated by (desired allocation percent - (no. allocated to that bucket / total allocated))
        train_displacement = train_split - (len(train_data) / assigned_count)
        val_displacement = val_split - (len(val_data) / assigned_count)
        test_displacement = test_split - (len(test_data) / assigned_count)
        
        # Allocate this patient to the bucket furthest from allocation (i.e. max displacement)
        if train_displacement > val_displacement and train_displacement > test_displacement:
            # Use extend so train_data is flat
            train_data.extend(patient_data[patient_id])
        elif val_displacement > test_displacement:
            val_data.extend(patient_data[patient_id])
        else:
            test_data.extend(patient_data[patient_id])
    
    return train_data, val_data, test_data

train_paths, val_paths, test_paths = train_val_test_split(0.85, 0.1, 0.05, patient_data)

# Sanity checks
source_data_length = len(data_paths)
assert len(train_paths) + len(val_paths) + len(test_paths) == source_data_length

print("Train Count: " + str(len(train_paths)))
print("Val Count: " + str(len(val_paths)))
print("Test Count: " + str(len(test_paths)))

Splitting dataset using Train: 0.85 Val: 0.1 Test: 0.05
Train Count: 177
Val Count: 24
Test Count: 10


In [98]:
# Generator:
# Maximum scan voxel value in dataset was above 512 but below 1023 so i use closest power of two -> 1023, for normalisation
class Prostate3DGenerator(keras.utils.Sequence):
    def __init__(self, data_paths, batch_size, data_dimensions, class_count):
        self.data_paths = data_paths
        self.batch_size = batch_size
        self.data_dimensions = data_dimensions
        self.class_count = class_count
        
    def __len__(self):
        return int(np.floor(len(self.data_paths) / self.batch_size))
    
    def __getitem__(self, index):
        start_index = index * self.batch_size
        batch_data_paths = self.data_paths[start_index : start_index + self.batch_size]
        
        scans = np.empty((self.batch_size, *self.data_dimensions))
        labels = np.empty((self.batch_size, *self.data_dimensions, self.class_count), dtype=int)
        
        for dataIndex in range(len(batch_data_paths)):
            # Populate "scans"
            #scan_voxels = nib.load(batch_data_paths[dataIndex][0])
            #scans[dataIndex] = tf.cast(np.array(scan_voxels.dataobj) / 1023.0, tf.float32)
            
            scan_voxels = nib.load(batch_data_paths[dataIndex][0])
            np_scan_voxels = np.array(scan_voxels.dataobj)[112:144, 112:144, 48:80]
            scans[dataIndex] = tf.cast(np_scan_voxels / 1023.0, tf.float32)
            
            # Populate "labels"
            #nibabel_voxels = nib.load(batch_data_paths[dataIndex][1])
            #prepared_voxels = tf.cast(np.array(nibabel_voxels.dataobj), tf.float32)
            #labels[dataIndex] = keras.utils.to_categorical(prepared_voxels, num_classes=self.class_count)
            
            nibabel_voxels = nib.load(batch_data_paths[dataIndex][1])
            np_nibabel_voxels = np.array(nibabel_voxels.dataobj)[112:144, 112:144, 48:80]
            prepared_voxels = tf.cast(np_nibabel_voxels, tf.float32)
            labels[dataIndex] = keras.utils.to_categorical(prepared_voxels, num_classes=self.class_count)
            
        return scans, labels

In [99]:
# Initialize generators
batch_size = 1

train_generator = Prostate3DGenerator(train_paths, batch_size, data_dimensions, class_count)
val_generator = Prostate3DGenerator(val_paths, batch_size, data_dimensions, class_count)
test_generator = Prostate3DGenerator(test_paths, batch_size, data_dimensions, class_count)

#item = train_generator.__getitem__(0)
#plt.imshow(item[0][0][15], interpolation='nearest')
#plt.show()

In [100]:
# Construct keras model
first_layer = layers.Input((*data_dimensions, 1))

# We will pass through "previous" to represent last layer
previous = first_layer

filters = [64, 128, 256]

# LEFT SIDE
downscale_layers = len(filters)
downscale_filters = filters
downscale_tails = []
for i in range(downscale_layers):
    previous = layers.Conv3D(downscale_filters[i], (3, 3, 3), padding='same', activation='relu')(previous)
    previous = layers.BatchNormalization()(previous)
    previous = layers.Conv3D(downscale_filters[i], (3, 3, 3), padding='same', activation='relu')(previous)
    
    if i != downscale_layers - 1:
        downscale_tails.append(previous)
        previous = layers.MaxPool3D((2, 2, 2), strides=(2, 2, 2))(previous)
        previous = layers.Dropout(0.2)(previous)
        
# RIGHT SIDE
# reverse references, since upscale is looped other way
downscale_tails = list(reversed(downscale_tails))

upscale_layers = len(filters) - 1
upscale_filters = list(reversed(filters))[1:]
for i in range(upscale_layers):
    # Up Convolution
    previous = layers.Conv3DTranspose(upscale_filters[i], (2, 2, 2), strides=(2, 2, 2))(previous)
    previous = layers.Dropout(0.2)(previous)
    
    # Pull across
    tail = downscale_tails[i]
    previous = layers.concatenate([previous, tail])
    
    # Convolutions
    previous = layers.Conv3D(upscale_filters[i], (3, 3, 3), padding='same', activation='relu')(previous)
    previous = layers.BatchNormalization()(previous)
    previous = layers.Conv3D(upscale_filters[i], (3, 3, 3), padding='same', activation='relu')(previous)
        
last_layer = layers.Conv3D(class_count, (1, 1, 1), activation='softmax')(previous)

model = keras.Model(first_layer, last_layer)

print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 32,  0                                            
__________________________________________________________________________________________________
conv3d_44 (Conv3D)              (None, 32, 32, 32, 6 1792        input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 32, 32, 32, 6 256         conv3d_44[0][0]                  
__________________________________________________________________________________________________
conv3d_45 (Conv3D)              (None, 32, 32, 32, 6 110656      batch_normalization_20[0][0]     
____________________________________________________________________________________________

In [103]:
# Loss Function and coefficients to be used during training:
def dice_coefficient(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    #y_pred = tf.cast(y_pred, tf.float32)
    
    smoothing_factor = 1
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    #print(flat_y_pred.shape)
    return (2. * K.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + smoothing_factor)

def dice_coefficient_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

#item = train_generator.__getitem__(0)
#dice_coefficient(item[1][0], item[1][0])

In [104]:
model.compile(optimizer="adam", loss=dice_coefficient_loss, metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint("oasis.h5", save_best_only=True)
]

# Train the model, doing validation at the end of each epoch.
epochs = 15

#model.load_weights("oasis.h5")
train_data = model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=callbacks)

Epoch 1/15
 33/177 [====>.........................] - ETA: 27s - loss: 0.4818 - accuracy: 0.5308

KeyboardInterrupt: 

In [66]:
model.evaluate(test_generator)

9/9 [==============================] - 2s 243ms/step - loss: 0.1908 - accuracy: 0.8094


[0.1907830834388733, 0.8094143271446228]